In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from latentcor import get_tps, latentcor
from gglasso.problem import glasso_problem
from matplotlib import pyplot as plt

In [ ]:
def plot_heatmap(cov, precision, mask, low_rank=None, low=False):

    left = 0.125  # the left side of the subplots of the figure
    right = 0.9  # the right side of the subplots of the figure
    bottom = 0.1  # the bottom of the subplots of the figure
    top = 0.9  # the top of the subplots of the figure
    wspace = -0.6  # the amount of width reserved for blank space between subplots,
    hspace = 0.5  # the amount of height reserved for white space between subplots,
    fontsize = 56
    cmap = "coolwarm"
    vmin = -0.5
    vmax = 0.5
    linewidth = .5
    square = True
    cbar = False

    if low:
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(90, 35))

            plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)

            ax1.get_shared_y_axes().join(ax2)
            ax3.get_shared_y_axes().join(ax4)

            g1 = sns.heatmap(cov, cmap=cmap, cbar=cbar, ax=ax1, vmin=vmin, vmax=vmax, linewidth=linewidth, square=square,
                             xticklabels=ticks, yticklabels=ticks)
            g1.set_ylabel('')
            g1.set_xlabel('Covariance', fontsize=fontsize)

            g2 = sns.heatmap(precision, cmap=cmap, cbar=cbar, ax=ax2, vmin=vmin, vmax=vmax, linewidth=linewidth, square=square,
                             xticklabels=ticks, yticklabels=ticks)
            g2.set_ylabel('')
            g2.set_xlabel('Inverse covariance', fontsize=fontsize)
            g2.set_yticks([])

            g3 = sns.heatmap(low_rank, cmap=cmap, ax=ax3, cbar=cbar, vmin=vmin, vmax=vmax, linewidth=linewidth, square=square,
                             xticklabels=ticks, yticklabels=ticks)
            g3.set_ylabel('')
            g3.set_xlabel('Low-rank solution', fontsize=fontsize)
            g3.set_yticks([])

            g4 = sns.heatmap(mask, cmap=cmap, ax=ax4, cbar=cbar, vmin=vmin, vmax=vmax, linewidth=linewidth, square=square,
                             xticklabels=ticks, yticklabels=ticks)
            g4.set_ylabel('')
            g4.set_xlabel('Mask', fontsize=fontsize)
            g4.set_yticks([])
    else:

        wspace = 0.5  # the amount of width reserved for blank space between subplots,
        hspace = 0.5

        fig, (ax1, ax2 ,ax3) = plt.subplots(1, 3, figsize=(90, 35))

        plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top, wspace=wspace, hspace=hspace)

        ax1.get_shared_y_axes().join(ax2, ax3)

        g1 = sns.heatmap(cov, cmap=cmap, cbar=cbar, ax=ax1, vmin=vmin, vmax=vmax, linewidth=linewidth, square=square,
                         xticklabels=ticks, yticklabels=ticks)
        g1.set_ylabel('')
        g1.set_xlabel('Covariance', fontsize=fontsize)

        g2 = sns.heatmap(precision, cmap=cmap, cbar=cbar, ax=ax2, vmin=vmin, vmax=vmax, linewidth=linewidth,
                         square=square,
                         xticklabels=ticks, yticklabels=ticks)
        g2.set_ylabel('')
        g2.set_xlabel('Inverse covariance', fontsize=fontsize)
        g2.set_yticks([])

        g3 = sns.heatmap(mask, cmap=cmap, ax=ax3, cbar=cbar, vmin=vmin, vmax=vmax, linewidth=linewidth, square=square,
                         xticklabels=ticks, yticklabels=ticks)
        g3.set_ylabel('')
        g3.set_xlabel('Mask', fontsize=fontsize)
        g3.set_yticks([])

    return fig

### Import data

In [ ]:
acm = pd.read_csv('~/GGLasso/data/soil/processed/acm_clr.tsv', sep='\t', index_col = 0)

meta = pd.read_csv('~/GGLasso/data/soil/processed/acm_meta.tsv', sep='\t', index_col = 0)
meta = meta.loc[:, meta.iloc[0, :] != 'categorical']
meta = meta.apply(pd.to_numeric, errors='coerce')
meta = meta.dropna(how='all')
meta = meta.iloc[1:]


df = acm.join(meta)
df = df.dropna(axis=0)

df.isnull().sum().any()
print(df.var().to_string())

In [ ]:
# no variance
del df['depth']

In [ ]:
N, p = df.shape

clean_types = get_tps(df)

In [ ]:
### N, p input
lat_cor = latentcor(df, tps=clean_types, method='original', use_nearPD=False)

In [ ]:
S = lat_cor['R']

meta_ticks = np.array(S.columns[-14:])
bug_ticks = np.arange(len(S.columns[:-14]))
ticks = np.hstack((bug_ticks, meta_ticks))

#### Solve SGL and SGL+low-rank with weighted solver

In [ ]:
lambda1_range = np.logspace(0.5, -1.5, 8)
mu1_range = np.logspace(1.5, -0.5, 6)

shape_meta = (p, p)
mask = np.zeros(shape_meta)
n_bugs = len(bug_ticks)
bugs_block = np.ones((n_bugs, n_bugs))
mask[0:n_bugs, 0:n_bugs] += bugs_block
lambda1_mask_exp = mask

lambda1_mask_abs = abs(S.values)

### SGL_mask of abs corr values - not optimal

In [ ]:
P_abs = glasso_problem(S.values, N, latent=True, do_scaling=False)
print(P_abs)

In [ ]:
modelselect_params = {'lambda1_range': lambda1_range, 'mu1_range': mu1_range, 'lambda1_mask': lambda1_mask_abs}

P_abs.model_selection(modelselect_params=modelselect_params, method='eBIC', gamma=0.25)
print(P_abs.reg_params)

Theta_abs = P_abs.solution.precision_

### SGL_mask_explicit - not optimal

In [ ]:
P_exp = glasso_problem(S.values, N, latent=True, do_scaling=False)
print(P_exp)

modelselect_params = {'lambda1_range': lambda1_range, 'mu1_range': mu1_range,
                      'lambda1_mask': lambda1_mask_exp}


P_exp.model_selection(modelselect_params=modelselect_params, method='eBIC', gamma=0.25)

print(P_exp.reg_params)

Theta_exp = P_exp.solution.precision_
L = P_exp.solution.lowrank_

### no low_rank exp - optimal

In [ ]:
P_exp_no = glasso_problem(S.values, N, latent=False, do_scaling=False)
print(P_exp_no)

modelselect_params = {'lambda1_range': lambda1_range, 'lambda1_mask': lambda1_mask_exp}

P_exp_no.model_selection(modelselect_params=modelselect_params, method='eBIC', gamma=0.25)

print(P_exp_no.reg_params)

Theta_exp_no = P_exp_no.solution.precision_

### no low_abs

In [ ]:
P_abs_no = glasso_problem(S.values, N, latent=False, do_scaling=False)
print(P_exp_no)

modelselect_params = {'lambda1_range': lambda1_range, 'lambda1_mask': lambda1_mask_abs}

P_abs_no.model_selection(modelselect_params=modelselect_params, method='eBIC', gamma=0.25)

print(P_abs_no.reg_params)
Theta_abs_no = P_abs_no.solution.precision_

### Save plots

In [ ]:
fig = plot_heatmap(cov=S, precision=Theta_exp, low_rank=L, mask=mask, low=True)
fig.savefig("../plots/low_rank_exp.png")

fig = plot_heatmap(cov=S, precision=Theta_abs, low_rank=L, mask=abs(S.values), low=True)
fig.savefig("../plots/low_rank_abscorr.png")


fig = plot_heatmap(cov=S, precision=Theta_exp_no, mask=mask, low=False)
fig.savefig("../plots/exp_no.png")

fig = plot_heatmap(cov=S, precision=Theta_abs_no, mask=abs(S.values), low=False)
fig.savefig("../plots/abscorr_no.png")